<a href="https://colab.research.google.com/github/AbdallhOmarr/solving-vrp-with-time-window/blob/master/vehicle_routing_problem_with_real_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ortools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# -*- coding: utf-8 -*-
"""learning or tools.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1vXgYappVyQEe3mPGnVf8bdUOVX3qCf_t
"""
import json


import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(json_path):
    """Stores the data for the problem."""
    # data = {}
    # # Locations in block units
    # data['locations'] = [(456, 320), # location 0 - the depot
    #                       (228, 0),    # location 1
    #                       (912, 0),    # location 2
    #                       (0, 80),     # location 3
    #                       (114, 80),   # location 4
    #                       (570, 160)]  # location 5]  # location 16 # yapf: disable
    # data['num_vehicles'] = 2
    # data['demands'] = [0, 1, 1, 2, 4, 0]
    # data['vehicle_capacities'] = [10,10]
    # data['starts'] = [0,0]
    # data['ends'] = [5,5]

    # Open the JSON file
    with open(json_path) as file:
        # Load the contents of the file
        data = json.load(file)

    return data

data = create_data_model("realdata.json")

def compute_scaled_distance_matrix(locations, scale_factor):
    """Creates callback to return scaled distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance scaled by the factor
                distance = math.hypot((from_node[0] - to_node[0]), (from_node[1] - to_node[1]))
                scaled_distance = int(distance * scale_factor)
                distances[from_counter][to_counter] = scaled_distance
    return distances

# Scale factor for distance
scale_factor = 100000

# Compute the scaled distance matrix
scaled_dist_matrix = compute_scaled_distance_matrix(data["locations"], scale_factor)

# Update the data dictionary with the scaled distance matrix
data['distance_matrix'] = scaled_dist_matrix

# Write the modified data back to the file
with open('realdata_with_scaled_distance_matrix.json', 'w') as file:
    json.dump(data, file, indent=4)




In [3]:
len(data['locations'])

71

In [4]:
len(data["distance_matrix"])

71

In [5]:
len(data['demands'])

71

In [6]:
data['num_vehicles']

33

In [7]:
len(data["starts"]),len(data['ends'])

(33, 33)

In [8]:
len(data['vehicle_capacities'])

33

In [9]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'],
                                           data['ends'])



In [10]:
routing = pywrapcp.RoutingModel(manager)


In [11]:

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


In [12]:

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    10,  # no slack
    60000000000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    30,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# search_parameters.local_search_metaheuristic = (
#     routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
# search_parameters.time_limit.seconds = 30
# search_parameters.log_search = True


In [ ]:

solution = routing.SolveWithParameters(search_parameters)
solution

In [ ]:


if solution:
    print_solution(data,manager, routing, solution)

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
  print('Route', i, route)